In [9]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

In [10]:
# Define paramaters for the model
learning_rate = 0.01
batch_size = 128
n_epochs = 100

Step 1: Read in data<br>
using TF Learn's built in function to load MNIST data to the folder data/mnist

In [11]:
mnist = input_data.read_data_sets('./data/mnist', one_hot=True) 

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


Step 2: create placeholders for features and labels<br>
each image in the MNIST data is of shape 28*28 = 784<br>
therefore, each image is represented with a 1x784 tensor<br>
there are 10 classes for each image, corresponding to digits 0 - 9.<br>
Features are of the type float, and labels are of the type int<br>

In [12]:
X = tf.placeholder(tf.float64, shape=(None, 784), name='X')
Y = tf.placeholder(tf.int32, shape=(None, 10), name='Y')

Step 3: create weights and bias<br>
weights and biases are initialized to 0<br>
shape of w depends on the dimension of X and Y so that Y = X * w + b<br>
shape of b depends on Y<br>

In [13]:
W = tf.Variable(tf.zeros([784, 10], dtype=tf.float64), dtype=tf.float64, name='W')
b = tf.Variable(tf.zeros([10], dtype=tf.float64), dtype=tf.float64, name='b')

Y_pred = tf.add(tf.matmul(X, W), b)

Step 4: build model<br>
the model that returns the logits.<br>
this logits will be later passed through softmax layer<br>
to get the probability distribution of possible label of the image<br>
DO NOT DO SOFTMAX HERE<br>

In [14]:
# logits = tf.log(tf.divide(Y_pred, 1.0 - Y_pred))
# logits = tf.sigmoid(Y_pred)
# logits = tf.log(Y_pred)
logits = Y_pred

Step 5: define loss function<br>
use cross entropy loss of the real labels with the softmax of logits<br>
use the method:<br>
tf.nn.softmax_cross_entropy_with_logits(logits, Y)<br>
then use tf.reduce_mean to get the mean loss of the batch<br>

In [15]:
def loss_function(logits, Y):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)
    return tf.reduce_mean(cross_entropy)

loss = loss_function(logits, Y)   
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

Step 6: define training op<br>
using gradient descent to minimize loss

In [16]:
with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())	
    n_batches = int(mnist.train.num_examples/batch_size)
    for i in range(n_epochs): # train the model n_epochs times
        total_loss = 0

        for _ in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            # TO-DO: run optimizer + fetch loss_batch
            
            _, loss_batch = sess.run([optimizer, loss], 
                                     feed_dict={X:X_batch, Y:Y_batch})
            
            
            total_loss += loss_batch
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))

    print('Total time: {0} seconds'.format(time.time() - start_time))

    print('Optimization Finished!') # should be around 0.35 after 25 epochs

    # test the model
    preds = tf.nn.softmax(logits)
    correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32)) # need numpy.count_nonzero(boolarr) :(

    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0
    
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        accuracy_batch = sess.run([accuracy], feed_dict={X: X_batch, Y:Y_batch}) 
        total_correct_preds += accuracy_batch[0]
        
    print('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))

Average loss epoch 0: 1.2889977731474276
Average loss epoch 1: 0.732583129592867
Average loss epoch 2: 0.6005408484317143
Average loss epoch 3: 0.5371560138220267
Average loss epoch 4: 0.49723696422324
Average loss epoch 5: 0.47108141468305603
Average loss epoch 6: 0.45133306069630136
Average loss epoch 7: 0.43604645600098185
Average loss epoch 8: 0.42288502736390554
Average loss epoch 9: 0.4137359276344566
Average loss epoch 10: 0.4044379568715026
Average loss epoch 11: 0.39639618987445824
Average loss epoch 12: 0.39043713938647484
Average loss epoch 13: 0.3849162410588992
Average loss epoch 14: 0.3790939226898735
Average loss epoch 15: 0.37403185041462406
Average loss epoch 16: 0.369837315088555
Average loss epoch 17: 0.3673731732265973
Average loss epoch 18: 0.363539382373347
Average loss epoch 19: 0.3590135424984655
Average loss epoch 20: 0.35665221292737426
Average loss epoch 21: 0.3532864774485642
Average loss epoch 22: 0.35133792180175455
Average loss epoch 23: 0.349631216398509